###  Implementation of a Low Pass FIR Filter with DMAs in SDSoC Ported to python

In [1]:
from pynq import Overlay,PL,Xlnk
from fir import general_const
import timeit
import time

#load bitstream
libfile = general_const.LIBRARY
bitfile = general_const.BITFILE
fir_overlay = Overlay(bitfile)     
if PL.bitfile_name != bitfile:
    fir_overlay.download()
    
xlnk = Xlnk()
def mem_init(buflen):
    """ Allocate contiguous memory buffer
    """
    buf = xlnk.cma_alloc(buflen)
    return xlnk.cma_cast(buf, "int")

number=2000
n = dlen = 1000
din = mem_init(n*4)
win = mem_init(85)
dout = mem_init(n*4)

### ABI

In [2]:
t1 = time.time()
import cffi
ffi0 = cffi.FFI()
ffi0.cdef("void _p0_cpp_FIR_1_noasync(int *x, int w[85], int *ret, "
                  "int datalen);")
lib0 = ffi0.dlopen(libfile)
t2 = time.time()

# call accelerator
def abiresp():
    lib0._p0_cpp_FIR_1_noasync(din,win,dout,dlen)

print("Running the benchmarks")
print("Time taken by abi", number,"times",timeit.timeit(abiresp,number=number))
print("Time taken by abi for import/init: ", t2-t1)

Running the benchmarks
Time taken by abi 2000 times 1.9080947819747962
Time taken by abi for import/init:  0.019701242446899414


### API

In [3]:
# Steps taken for API mode:
#   1. run example_build.py
#   2. add "from _example import fir, lib" to __init__.py

t1 = time.time()
from fir import ffi, lib
t2 = time.time()

# call accelerator
def apiresp():
    lib._p0_cpp_FIR_1_noasync(din,win,dout,dlen)

print("Running the benchmarks")
print("Time taken by api", number,"times",timeit.timeit(apiresp,number=number))
print("Time taken by api for import/init: ", t2-t1)

Running the benchmarks
Time taken by api 2000 times 1.9013586430228315
Time taken by api for import/init:  0.0005807876586914062
